In [1]:
from utils_1.goenv import GoEnv,_format_state
from utils_1.fnet import NeuralTrainer, AlphaNeural
import numpy as np
import goSim

In [16]:
class AlphaGoPlayer():
    def __init__(self, init_state, seed, player_color):
        # setup parameters
        self.init_state = init_state
        self.seed = seed
        self.player_color = player_color
        
        # init environment
        if player_color==1:
            self.env = GoEnv(player_color="black", board_size=13)
        else:
            self.env = GoEnv(player_color="white", board_size=13)
        self.env.reset()
        
        # load neural network
        self.model = NeuralTrainer(res_blocks=10, board_size=13)
#         self.model.load_model("utils_1/networks/testing1.model")
        
    def get_action(self, cur_state, opponent_action):
        if opponent_action>=0:
            self.env.step(opponent_action)
            print('-------------------------------------')
            print('opponent_action: ' + str(opponent_action))
            print('-------------------------------------')
        
        policy, value = self.model.predict(self.env.get_history())
        legal_moves = self.env.get_legal_moves()
        policy = policy*legal_moves
        action = np.argmax(policy)
        self.env.step(action)
        
        print('-------------------------------------')
        print('player_action: ' + str(action))
        print('-------------------------------------')
        self.env.print_board()
        
        return action

In [3]:
env = goSim.GoEnv(player_color="black", observation_type='image3c', illegal_move_mode="raise", board_size=13, komi=7.5)

obs_t = env.reset()
opp_action=-1

In [4]:
a = AlphaGoPlayer(obs_t.copy(), 1, 1)

In [5]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)

-------------------------------------
player_action: 140
-------------------------------------
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . X . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
140


In [6]:
env.set_player_color(2)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

13


In [7]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)

-------------------------------------
opponent_action: 13
-------------------------------------
-------------------------------------
player_action: 32
-------------------------------------
. . . . . . . . . . . . . 
O . . . . . . . . . . . . 
. . . . . . X . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . X . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
32


In [8]:
env.set_player_color(2)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

29


In [9]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)

-------------------------------------
opponent_action: 29
-------------------------------------
-------------------------------------
player_action: 83
-------------------------------------
. . . . . . . . . . . . . 
O . . . . . . . . . . . . 
. . . O . . X . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . X . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . X . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
83


In [10]:
env = goSim.GoEnv(player_color="black", observation_type='image3c', illegal_move_mode="raise", board_size=13, komi=7.5)
obs_t = env.reset()
opp_action=-1
a = AlphaGoPlayer(obs_t.copy(), 2, 1)


env.set_player_color(1)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

147


In [11]:
env.set_player_color(2)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)

-------------------------------------
opponent_action: 147
-------------------------------------
-------------------------------------
player_action: 161
-------------------------------------
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . X . . . . . . . . 
. . . . . O . . . . . . . 
161


In [12]:
env.set_player_color(1)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

env.set_player_color(2)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)


60
-------------------------------------
opponent_action: 60
-------------------------------------
-------------------------------------
player_action: 136
-------------------------------------
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . X . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . O . . . . . . 
. . . . X . . . . . . . . 
. . . . . O . . . . . . . 
136


In [13]:
fnet = NeuralTrainer(res_blocks=10, board_size=13)

In [14]:
env_mank = GoEnv("black", board_size=13)
env_mank.reset();

In [24]:
for i in range(100):
    s = env_mank.get_history()
    policy, value = fnet.predict(s)
    policy = env_mank.get_legal_moves()*policy
    env_mank.step(np.argmax(policy))
    env_mank.print_board()
    print(np.argmax(policy), value)
    legal_moves = np.nonzero(env_mank.get_legal_moves())[0]
    opp_action = np.random.choice(legal_moves)
    env_mank.step(opp_action)
    print(opp_action)

. . X . . . . . . . O . . 
. . O . X . . . . . . . . 
. . . . . . . . O . . . . 
. . . . . . . . . . X . . 
. . . . . . . . . . . . . 
X . . . . . . . . . . . . 
. . . . . . . . . X . . . 
. . . . . . . X . . . . X 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . O . . . . . . 
. . . O . . O . . . . . . 
65 [0.04316535]
60
. . X . . . . . . . O . . 
. . O . X . . . . . . . . 
. . . . . . . . O . . . . 
. . . . . . . . . . X . X 
. . . . . . . . O . . . . 
X . . . . . . . . . . . . 
. . . . . . . . . X . . . 
. . . . . . . X . . . . X 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . O . . . . . . 
. . . O . . O . . . . . . 
51 [0.04035366]
54
. . X . . . . . . . O . . 
. . O . X . . . . . . . . 
. . . . . . . . O . . . . 
. . . . . . . . . . X . X 
. . O . . . . . O . X . . 
X . . . . . . . . . . . . 
. . . . . . . . . X . . . 
. . . . . . . X . . . . X 
. . . . . . . . . . . . . 
. . . . . . . . .

128
. X X X . . . O X O O . O 
. X O . X . . X . . . . . 
O . O . . . O O O O . . O 
. O O . . . . . . . X . X 
. . O . . . . X O O X . . 
X . . . X . O X X X . . X 
. X O O X . O . . X . X . 
O X . . . . . X . . X . X 
. . . . . . . . . X . . . 
. . O X . . . X X . X O . 
. O . O . . . . . X X O . 
. . . . . O O . O . . O . 
X . . O O . O . . . X . X 
89 [0.04975597]
158
. X X X . . . O X O O . O 
. X O . X . . X . . . . . 
O . O . . . O O O O . . O 
. O O . . . . . . . X . X 
. . O . . . . X O O X . . 
X . . . X . O X X X . . X 
. X O O X . O . . X . X . 
O X . . . . . X . . X . X 
. . . . . . . . . X . . . 
. . O X . X . X X . X O . 
. O . O . . . . . X X O . 
. . . . . O O . O . . O . 
X . O O O . O . . . X . X 
122 [0.04350606]
29
. X X X . . . O X O O . O 
. X O . X . . X . . . . . 
O . O O . . O O O O . . O 
. O O . . . . . . . X . X 
. . O . . . . X O O X . . 
X . . . X . O X X X . . X 
. X O O X . O . . X . X . 
O X . . . . . X . . X . X 
. . . . . . . . . X . . . 
. . O X . X

X O . O . O X O . X X O . 
. O X O O O O . O O O O . 
X . O O O O O O X X X X X 
165 [0.05686137]
123
O X X X X . O O X O O X O 
O X O . X O X X . X . X . 
O X O O . . O O O O O X O 
. O O . X X . O X . X . X 
. . O O O O . X O O X . O 
X X . . X . O X X X . X X 
X X O O X X O . . X X X X 
O X . O . X O X O . X . X 
. . . . X O . O . X X X . 
O X O X X X O X X . X O O 
X O . O . O . O . X X O . 
. O X O O O O . O O O O . 
X . O O O O O O X X X X X 
44 [0.04872774]
21
O X X X X . O O . O O X O 
O X O . X O . X O X . X . 
O X O O . . O O O O O X O 
. O O . X X . O X . X . X 
. . O O O O . X O O X . O 
X X . . X . O X X X . X X 
X X O O X X O . . X X X X 
O X . O . X O X O . X . X 
. . . . X O . O . X X X . 
O X O X X X O X X . X O O 
X O . O . O . O . X X O . 
. O X O O O O . O O O O . 
X . O O O O O O X X X X X 
20 [0.04646704]
132
O X X X X . O O . O O X O 
O X O . X O . X O X . X . 
O X O O . . O O O O O X O 
. O O . X X . O X . X . X 
. . O O O O . X O O X . O 
X X . . X . O X X X . 

O X X X X X O O O O O X X 
O X O O X O O O O X . X X 
O X O O . O O O O O O X . 
O O O . X X . O X X X O X 
O . O O O O X X X X X . O 
X . O O X O O X X X X X X 
. . O O X X O O X X X X X 
O X O O O X O X X X X X X 
O . O O X . X X X X X X O 
O . O X X X . X X . X O O 
O O O O O O X O X X X O . 
O O O O O O O . O O O O X 
X . O O O O O O X X X X X 
92 [0.03997612]
66
O X X X X X O O O O O X X 
O X O O X O O O O X . X X 
O X O O . O O O O O O X . 
O O O . X X . O X X X O X 
O . O O O O X X X X X . O 
X O O O X O O X X X X X X 
. X O O X X O O X X X X X 
O X O O O X O X X X X X X 
O . O O X . X X X X X X O 
O . O X X X . X X . X O O 
O O O O O O X O X X X O . 
O O O O O O O . O O O O X 
X . O O O O O O X X X X X 
79 [0.03281884]
118
O X X X X X O O O O O X X 
O X O O X O O O O X . X X 
O X O O . O O O O O O X . 
O O O . X X . O X X X O X 
O . O O O O X X X X X . O 
X O O O X O O X X X X X X 
X X O O X X O O X X X X X 
O X O O O X O X X X X X X 
O . O O X . X X X X X X O 
O O O X X X . X 